# Adaptive Tax 

This notebook explores how outcomes can be improved by adopting adaptive tax rules. We explore four variant across three different tax adjustment rules and examine how they compare to static tax levels in terms of outcomes.

For each variant of adjustment rule we:
1. Simulate 10 years with a time-varying carbon tax, $\tau(t)$.
2. Compute time-averaged tax over 10 years (mean of monthly taxes), cumulative CO₂ at t=120, cumulative profit at t=120, and viability.
3. Compare that to a static tax equal to the same average level:
 
   $$\bar{\tau} = \frac{1}{T} \sum_{t=1}^T{\tau(t)}$$ 
 
4. Run the model once with `carbon_tax_rate` $=\bar{\tau}$ as a constant and determine if the adaptive tax achieves higher cumulative profit for similar emissions, lower emissions for similar profit or both (best case, this means it's Pareto-better).

The adaptive tax rules that will be examined is:
1. Simple annual step increase:

   $$\tau_{t+12} = \min(\tau_t + \delta_{\text{step}}, \tau_{\max})$$

   Tax climbs by a fixed amount each year until it reaches a cap mimicing a pre-announced tax ramp.

2. Percentage growth rule: 

   $$\tau_{t+12} = \tau_t(1 + g)$$

   Tax grows at g% per year mimicing indexing tax to some growth rule.

3. Profit / margin-based rule:

   $$
   \tau_{t+12} = \begin{cases}
   \tau_t + \Delta_{\uparrow} & \text{if } \text{rolling\_margin}_t > m^* + \text{band} \\
   \tau_t & \text{if } m^* - \text{band} \leq \text{rolling\_margin}_t \leq m^* + \text{band} \\
   \tau_t - \Delta_{\downarrow} & \text{if } \text{rolling\_margin}_t < m^* - \text{band}
   \end{cases}
   $$

   This rule raises tax when there is 'room' (high margin), freezes or eases if operators are under stress (low margin).

## Configuration

In [ ]:
from pathlib import Path
import os
import pysd
import yaml
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

current_dir = Path.cwd()
BASE_DIR = current_dir.parent if current_dir.name == 'notebooks' else current_dir
os.chdir(BASE_DIR)

from src.utils.max_tax import calculate_max_viable_tax
from src.utils.tax_adjustment import compare_adaptive_tax_vs_static

# Load the default parameters
with open(BASE_DIR / 'params.yaml', 'r') as file:
    base_params = yaml.safe_load(file)

model = pysd.load('src/model.py')

print(f"Using model {BASE_DIR}/src/model.py with params {base_params}")

# Model tax frontier
tax_frontier = calculate_max_viable_tax(model, base_params)

print(f"Max viable tax for static tax: {tax_frontier}")

## Simple Annual Step Increase

### Define Rule Variants

In [ ]:
# Variant A explores CO2 and Profit outcomes
# This set of parameters increases the tax by 5 yen per year
r1_vA_params = base_params.copy()
r1_vA_params["carbon_tax_rate"] = 2500
r1_vA_step_size = 5
r1_vA_max_tax = tax_frontier

# Variant B explores CO2 and Profit outcomes
# This set of parameters increases the tax by 120 yen per year
r1_vB_params = base_params.copy()
r1_vB_params["carbon_tax_rate"] = 2500
r1_vB_step_size = 120
r1_vB_max_tax = tax_frontier

# Variant C explores viability when pushing the tax frontier
r1_vC_params = base_params.copy()
r1_vC_params["carbon_tax_rate"] = tax_frontier
r1_vC_step_size = 131
r1_vC_max_tax = tax_frontier * 1.5

# Variant D explores viability when pushing the tax frontier
r1_vD_params = base_params.copy()
r1_vD_params["carbon_tax_rate"] = tax_frontier
r1_vD_step_size = 132
r1_vD_max_tax = tax_frontier * 1.5


### Run Rule Variants

In [ ]:
from src.utils.tax_adjustment import step_increase_rule

# Variant A
result_r1_vA = compare_adaptive_tax_vs_static(
    model,
    r1_vA_params,
    step_increase_rule,
    step_size=r1_vA_step_size,
    max_tax=r1_vA_max_tax
)

# Variant B
result_r1_vB = compare_adaptive_tax_vs_static(
    model,
    r1_vB_params,
    step_increase_rule,
    step_size=r1_vB_step_size,
    max_tax=r1_vB_max_tax
)

# Variant C
result_r1_vC = compare_adaptive_tax_vs_static(
    model,
    r1_vC_params,
    step_increase_rule,
    step_size=r1_vC_step_size,
    max_tax=r1_vC_max_tax
)

# Variant D
result_r1_vD = compare_adaptive_tax_vs_static(
    model,
    r1_vD_params,
    step_increase_rule,
    step_size=r1_vD_step_size,
    max_tax=r1_vD_max_tax
)


### Summary Table

In [ ]:
# Create comparison table
comparison_table = pd.DataFrame({
    "Variant": ["A", "B", "C", "D"],
    "step size": [r1_vA_step_size, r1_vB_step_size, r1_vC_step_size, r1_vD_step_size],
    "max tax": [r1_vA_max_tax, r1_vB_max_tax, r1_vC_max_tax, r1_vD_max_tax],
    "avg tax": [
        result_r1_vA["time_avg_tax"],
        result_r1_vB["time_avg_tax"],
        result_r1_vC["time_avg_tax"],
        result_r1_vD["time_avg_tax"]
    ],
    "relative co2 diff": [
        result_r1_vA["co2_diff_pct"],
        result_r1_vB["co2_diff_pct"],
        result_r1_vC["co2_diff_pct"],
        result_r1_vD["co2_diff_pct"]
    ],
    "relative profit diff": [
        result_r1_vA["profit_diff_pct"],
        result_r1_vB["profit_diff_pct"],
        result_r1_vC["profit_diff_pct"],
        result_r1_vD["profit_diff_pct"]
    ],
    "viability static": [
        result_r1_vA["viability_static"],
        result_r1_vB["viability_static"],
        result_r1_vC["viability_static"],
        result_r1_vD["viability_static"]
    ],
    "viability rule": [
        result_r1_vA["viability_adaptive"],
        result_r1_vB["viability_adaptive"],
        result_r1_vC["viability_adaptive"],
        result_r1_vD["viability_adaptive"]
    ]
})

# Display the table
comparison_table

## Percentage Growth Rule

### Define Rule Variants

In [ ]:
# Variant A explores CO2 and Profit outcomes
# This set of parameters increases the tax by 1% per year
r2_vA_params = base_params.copy()
r2_vA_params["carbon_tax_rate"] = 2500
r2_vA_annual_growth_rate = 0.01

# Variant B explores CO2 and Profit outcomes
# This set of parameters increases the tax by 5% per year
r2_vB_params = base_params.copy()
r2_vB_params["carbon_tax_rate"] = 2500
r2_vB_annual_growth_rate = 0.05

# Variant C explores viability when achieving higher average tax than static tax frontier
r2_vC_params = base_params.copy()
r2_vC_params["carbon_tax_rate"] = tax_frontier
r2_vC_annual_growth_rate = 0.027

# Variant D explores viability when achieving higher average tax than static tax frontier
r2_vD_params = base_params.copy()
r2_vD_params["carbon_tax_rate"] = tax_frontier
r2_vD_annual_growth_rate = 0.028

### Run Variants

In [ ]:
from src.utils.tax_adjustment import percentage_growth_rule

# Variant A
result_r2_vA = compare_adaptive_tax_vs_static(
    model,
    r2_vA_params,
    percentage_growth_rule,
    annual_growth_rate=r2_vA_annual_growth_rate
)

# Variant B
result_r2_vB = compare_adaptive_tax_vs_static(
    model,
    r2_vB_params,
    percentage_growth_rule,
    annual_growth_rate=r2_vB_annual_growth_rate
)

# Variant C
result_r2_vC = compare_adaptive_tax_vs_static(
    model,
    r2_vC_params,
    percentage_growth_rule,
    annual_growth_rate=r2_vC_annual_growth_rate
)

# Variant D
result_r2_vD = compare_adaptive_tax_vs_static(
    model,
    r2_vD_params,
    percentage_growth_rule,
    annual_growth_rate=r2_vD_annual_growth_rate
)

### Summary Table

In [ ]:
# Create comparison table
comparison_table = pd.DataFrame({
    "Variant": ["A", "B", "C", "D"],
    "annual growth rate": [r2_vA_annual_growth_rate, r2_vB_annual_growth_rate, r2_vC_annual_growth_rate, r2_vD_annual_growth_rate],
    "avg tax": [
        result_r2_vA["time_avg_tax"],
        result_r2_vB["time_avg_tax"],
        result_r2_vC["time_avg_tax"],
        result_r2_vD["time_avg_tax"]
    ],
    "relative co2 diff": [
        result_r2_vA["co2_diff_pct"],
        result_r2_vB["co2_diff_pct"],
        result_r2_vC["co2_diff_pct"],
        result_r2_vD["co2_diff_pct"]
    ],
    "relative profit diff": [
        result_r2_vA["profit_diff_pct"],
        result_r2_vB["profit_diff_pct"],
        result_r2_vC["profit_diff_pct"],
        result_r2_vD["profit_diff_pct"]
    ],
    "viability static": [
        result_r2_vA["viability_static"],
        result_r2_vB["viability_static"],
        result_r2_vC["viability_static"],
        result_r2_vD["viability_static"]
    ],
    "viability rule": [
        result_r2_vA["viability_adaptive"],
        result_r2_vB["viability_adaptive"],
        result_r2_vC["viability_adaptive"],
        result_r2_vD["viability_adaptive"]
    ]
})

# Display the table
comparison_table

## Margin-based Rule

### Rule Variants

In [ ]:
# Variant A
r3_vA_params = base_params.copy()
r3_vA_params["carbon_tax_rate"] = 2500
r3_vA_target_margin = 0.02
r3_vA_margin_band = 0.005
r3_vA_tax_increase = 100
r3_vA_tax_decrease = 100

# Variant B
r3_vB_params = base_params.copy()
r3_vB_params["carbon_tax_rate"] = 2500
r3_vB_target_margin = 0.02
r3_vB_margin_band = 0.005
r3_vB_tax_increase = 300
r3_vB_tax_decrease = 200

# Variant C
r3_vC_params = base_params.copy()
r3_vC_params["carbon_tax_rate"] = tax_frontier
r3_vC_target_margin = 0.04
r3_vC_margin_band = 0.005
r3_vC_tax_increase = 600
r3_vC_tax_decrease = 300

# Variant D
r3_vD_params = base_params.copy()
r3_vD_params["carbon_tax_rate"] = tax_frontier
r3_vD_target_margin = 0.02
r3_vD_margin_band = 0.005
r3_vD_tax_increase = 400
r3_vD_tax_decrease = 300

### Run Variants

In [ ]:
from src.utils.tax_adjustment import margin_based_rule

# Variant A
result_r3_vA = compare_adaptive_tax_vs_static(
    model,
    r3_vA_params,
    margin_based_rule,
    target_margin=r3_vA_target_margin,
    margin_band=r3_vA_margin_band,
    tax_increase=r3_vA_tax_increase,
    tax_decrease=r3_vA_tax_decrease
)

# Variant B
result_r3_vB = compare_adaptive_tax_vs_static(
    model,
    r3_vB_params,
    margin_based_rule,
    target_margin=r3_vB_target_margin,
    margin_band=r3_vB_margin_band,
    tax_increase=r3_vB_tax_increase,
    tax_decrease=r3_vB_tax_decrease
)

# Variant C
result_r3_vC = compare_adaptive_tax_vs_static(
    model,
    r3_vC_params,
    margin_based_rule,
    target_margin=r3_vC_target_margin,
    margin_band=r3_vC_margin_band,
    tax_increase=r3_vC_tax_increase,
    tax_decrease=r3_vC_tax_decrease
)

# Variant D
result_r3_vD = compare_adaptive_tax_vs_static(
    model,
    r3_vD_params,
    margin_based_rule,
    target_margin=r3_vD_target_margin,
    margin_band=r3_vD_margin_band,
    tax_increase=r3_vD_tax_increase,
    tax_decrease=r3_vD_tax_decrease
)

### Summary Table

In [ ]:
comparison_table_R3 = pd.DataFrame({
    "Variant": ["A", "B", "C", "D"],
    "target margin": [r3_vA_target_margin, r3_vB_target_margin, r3_vC_target_margin, r3_vD_target_margin],
    "margin band": [r3_vA_margin_band, r3_vB_margin_band, r3_vC_margin_band, r3_vD_margin_band],
    "tax increase": [r3_vA_tax_increase, r3_vB_tax_increase, r3_vC_tax_increase, r3_vD_tax_increase],
    "tax decrease": [r3_vA_tax_decrease, r3_vB_tax_decrease, r3_vC_tax_decrease, r3_vD_tax_decrease],
    "avg tax": [
        result_r3_vA["time_avg_tax"],
        result_r3_vB["time_avg_tax"],
        result_r3_vC["time_avg_tax"],
        result_r3_vD["time_avg_tax"]
    ],
    "relative co2 diff": [
        result_r3_vA["co2_diff_pct"],
        result_r3_vB["co2_diff_pct"],
        result_r3_vC["co2_diff_pct"],
        result_r3_vD["co2_diff_pct"]
    ],
    "relative profit diff": [
        result_r3_vA["profit_diff_pct"],
        result_r3_vB["profit_diff_pct"],
        result_r3_vC["profit_diff_pct"],
        result_r3_vD["profit_diff_pct"]
    ],
    "viability static": [
        result_r3_vA["viability_static"],
        result_r3_vB["viability_static"],
        result_r3_vC["viability_static"],
        result_r3_vD["viability_static"]
    ],
    "viability rule": [
        result_r3_vA["viability_adaptive"],
        result_r3_vB["viability_adaptive"],
        result_r3_vC["viability_adaptive"],
        result_r3_vD["viability_adaptive"]
    ]
})


comparison_table_R3

## Rule Comparison

In [ ]:
rules = [
    "Static",
    "Step increase",
    "Percentage growth",
    "Margin-based"
]

max_viable_avg_tax = [
    tax_frontier,
    5570,
    5632,
    6691
]

# Position reference bar to the left, then rules with spacing
x_reference = 0  # Reference bar position
x_rules = np.arange(1, len(rules))  # Rules start at position 1

# Combine x positions
x_positions = [x_reference] + list(x_rules)

colors = [
    "#7f7f7f",   # Static (reference) - grey
    "#1f77b4",   # Step increase - blue
    "#1f77b4",   # Percentage growth - blue
    "#1f77b4"    # Margin-based - blue
]

hatches = [
    None,        # Static (reference)
    None,        # Step increase (not Pareto-better)
    None,        # Percentage growth (not Pareto-better)
    "//"         # Margin-based (Pareto-better) - only this one has hatch
]

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(
    x_positions,
    max_viable_avg_tax,
    color=colors,
    hatch=None,
    edgecolor="black",
    width=0.4
)

# Apply hatching selectively
for bar, hatch in zip(bars, hatches):
    if hatch:
        bar.set_hatch(hatch)

# Add value labels on top of each bar
for i, (bar, value) in enumerate(zip(bars, max_viable_avg_tax)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2., height,
            f'{value:.0f}',
            ha='center', va='bottom')

# Set x-ticks and labels
ax.set_xticks(x_positions)
ax.set_xticklabels(rules)

# Add vertical line to separate reference area
# ax.axvline(x=0.5, color='gray', linestyle='--', linewidth=1.5, alpha=0.5)

ax.set_ylabel("Maximum viable average carbon tax (¥/tCO₂)")
ax.set_title("Maximum feasible carbon tax under different adjustment rules")

from matplotlib.patches import Patch

legend_elements = [
    Patch(facecolor="#7f7f7f", edgecolor="black",
          label="Static tax (reference)"),
    Patch(facecolor="#1f77b4", edgecolor="black",
          label="Higher tax frontier is possible to achieve\nbut not Pareto-better outcomes"),
    Patch(facecolor="#1f77b4", edgecolor="black", hatch="//",
          label="Higher tax frontier and Pareto-better \noutcomes is possible to achieve")
]

ax.legend(
    handles=legend_elements,
    frameon=False,
    loc="upper left"
)


plt.tight_layout()
plt.savefig(f"{BASE_DIR}/figures/results/rule_comparison.png", dpi=300)
plt.show()